Load the earthquake dataset using a library like pandas in 
python.

In [6]:
import pandas as pd

# Replace 'your_dataset.csv' with the path to your earthquake dataset file
file_path = 'merge.csv'

# Load the dataset
earthquake_data = pd.read_csv(file_path)

# Display the first few rows of the dataframe
print(earthquake_data.head())
print(earthquake_data.info())

C:\Users\Admin\AppData\Local\Temp\ipykernel_18952\3439843406.py:7: DtypeWarning: Columns (7,11,13,14,24,25,26,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  earthquake_data = pd.read_csv(file_path)


  network_code receiver_code receiver_type  receiver_latitude  \
0           TA          109C            HH            32.8889   
1           TA          109C            HH            32.8889   
2           TA          109C            HH            32.8889   
3           TA          109C            HH            32.8889   
4           TA          109C            HH            32.8889   

   receiver_longitude  receiver_elevation_m  p_arrival_sample p_status  \
0           -117.1051                 150.0               NaN      NaN   
1           -117.1051                 150.0               NaN      NaN   
2           -117.1051                 150.0               NaN      NaN   
3           -117.1051                 150.0               NaN      NaN   
4           -117.1051                 150.0               NaN      NaN   

   p_weight  p_travel_sec  ...  source_magnitude_author  \
0       NaN           NaN  ...                      NaN   
1       NaN           NaN  ...                

Separate the feature attributes like latitude, 
longitude, depth, hour and the target attribute magnitude.... but the columns of the dataset are: 

In [7]:
import pandas as pd

# Assuming earthquake_data is your DataFrame
# Correct any column name discrepancies and ensure you've loaded the data correctly.

# Convert 'source_origin_time' to datetime format and extract the hour directly into the features DataFrame
earthquake_data['hour'] = pd.to_datetime(earthquake_data['source_origin_time']).dt.hour

# Now select your features including the newly created 'hour'
features = earthquake_data[['source_latitude', 'source_longitude', 'source_depth_km', 'hour']]

# Target remains the same
target = earthquake_data['source_magnitude']

print("Features Preview:")
print(features.head())
print("\nTarget Preview:")
print(target.head())


Features Preview:
   source_latitude  source_longitude  source_depth_km  hour
0              NaN               NaN              NaN   NaN
1              NaN               NaN              NaN   NaN
2              NaN               NaN              NaN   NaN
3              NaN               NaN              NaN   NaN
4              NaN               NaN              NaN   NaN

Target Preview:
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: source_magnitude, dtype: float64


Drop missing values from the dataset before separating features and the target.

In [8]:
import pandas as pd

# Assuming earthquake_data is your DataFrame
# Step 1: Drop rows with any missing values in the specified columns
cleaned_data = earthquake_data.dropna(subset=['source_latitude', 'source_longitude', 'source_depth_km', 'source_origin_time', 'source_magnitude'])

# Now that the data is cleaned, you can safely extract the hour from 'source_origin_time'
# Convert 'source_origin_time' to datetime and extract the hour
cleaned_data['hour'] = pd.to_datetime(cleaned_data['source_origin_time']).dt.hour

# Step 2: Separate features and target in the cleaned dataset
features = cleaned_data[['source_latitude', 'source_longitude', 'source_depth_km', 'hour']]
target = cleaned_data['source_magnitude']

print("Features Preview:")
print(features.head())
print("\nTarget Preview:")
print(target.head())


Features Preview:
        source_latitude  source_longitude  source_depth_km  hour
235426          33.7496         -117.4938             0.45    15
235427          32.7077         -116.0446             9.20    15
235428          32.7253         -116.0348            12.66    16
235429          32.7063         -116.0241            11.50    13
235430          31.9679         -117.1944             7.26    10

Target Preview:
235426    3.6
235427    4.3
235428    3.6
235429    3.8
235430    3.6
Name: source_magnitude, dtype: float64


C:\Users\Admin\AppData\Local\Temp\ipykernel_18952\1281162062.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['hour'] = pd.to_datetime(cleaned_data['source_origin_time']).dt.hour


The StandardScaler function is used to normalize the data. 

In [9]:
from sklearn.preprocessing import StandardScaler

# Assuming 'features' is your DataFrame containing the feature attributes
scaler = StandardScaler()

# Fit the scaler to your data and transform it
normalized_features = scaler.fit_transform(features)

# If you want to convert it back to a DataFrame
normalized_features_df = pd.DataFrame(normalized_features, columns=features.columns)

print("Normalized Features Preview:")
print(normalized_features_df.head())


Normalized Features Preview:
   source_latitude  source_longitude  source_depth_km      hour
0        -0.486272         -0.018463        -0.627976  0.550654
1        -0.561782          0.016048        -0.266589  0.550654
2        -0.560506          0.016281        -0.123686  0.697211
3        -0.561883          0.016536        -0.171596  0.257540
4        -0.615397         -0.011333        -0.346714 -0.182130


Then the dataset separate the dataset into training and testing 
sets using train_test_split function.

In [10]:
from sklearn.model_selection import train_test_split

# Assuming 'normalized_features' and 'target' are your features and target variables
# Split the dataset into 80% training and 20% testing, you can adjust the 'test_size' parameter as needed
X_train, X_test, y_train, y_test = train_test_split(normalized_features, target, test_size=0.2, random_state=42)

# Now you have X_train (features for training), X_test (features for testing),
# y_train (target for training), and y_test (target for testing)

# Display the shapes of the resulting sets
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (824145, 4)
Shape of X_test: (206037, 4)
Shape of y_train: (824145,)
Shape of y_test: (206037,)


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GraphConv

# Assuming you have already loaded and pre-processed your data into X_train, y_train, X_test, y_test

# Convert features and target to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Convert the edge_index to PyTorch tensor with shape [2, num_edges]
# For simplicity, assuming a fully connected graph, adjust accordingly based on your actual graph structure
num_nodes = X_train_tensor.shape[0]
edge_index_tensor = torch.tensor([
    [i for i in range(num_nodes) for _ in range(num_nodes)],
    [j for _ in range(num_nodes) for j in range(num_nodes)]
], dtype=torch.long)

# Define a simple GNN model using GraphConv layers
class GNNModel(nn.Module):
    def __init__(self, input_size):
        super(GNNModel, self).__init__()
        self.conv1 = GraphConv(input_size, 16)
        self.conv2 = GraphConv(16, 1)

    def forward(self, x, edge_index):
        x = torch.relu(self.conv1(x, edge_index))
        x = torch.relu(self.conv2(x, edge_index))
        return x

# Create a GNN model
input_size = X_train_tensor.shape[1]
model = GNNModel(input_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert data to PyTorch Geometric Data objects
train_data = Data(x=X_train_tensor, edge_index=edge_index_tensor, y=y_train_tensor)
test_data = Data(x=X_test_tensor, edge_index=edge_index_tensor, y=y_test_tensor)

# Define data loaders
train_loader = DataLoader([train_data], batch_size=64, shuffle=True)
test_loader = DataLoader([test_data], batch_size=64, shuffle=False)

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y.view(-1, 1))
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    for batch in test_loader:
        predictions = model(batch.x, batch.edge_index)
        test_loss = criterion(predictions, batch.y.view(-1, 1))

print("Test Loss:", test_loss.item())

MemoryError: 

In [16]:
pip install torch-geometric

Note: you may need to restart the kernel to use updated packages.
